In [2]:
from evalplus.data import (
    get_human_eval_plus,
    get_human_eval_plus_hash,
    get_mbpp_plus,
    get_mbpp_plus_hash,
    load_solutions,
)

from datasets import load_dataset

/mnt/data/haausing/miniconda3/envs/evalplus/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("evalplus/mbppplus")
problems = get_mbpp_plus()
for elem in dataset["test"]:
    task_id = "Mbpp/{}".format(elem["task_id"])
    problems[task_id]["text"] = elem["prompt"]

In [7]:
# load json file
import json
import pandas as pd
def load_json(dir_path: str):
    with open(dir_path, "r") as f:
        return json.load(f)
    
eval_results = load_json("../../evalplus_outputs/mbpp/deepseek-coder-7b-instruct-v1.5_temp_0.8/eval_results.json")
for task_id in eval_results["eval"]:
    eval_results["eval"][task_id] = sorted(eval_results["eval"][task_id], key=lambda x: int(x["solution_id"]))
log_probs = pd.read_pickle("/mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/mbpp/deepseek-coder-7b-instruct-v1.5_temp_0.8/logprobs.pkl")


In [46]:
#['code', 'exec_match', 'linear_score', 'exec_result', 'gen_prob', 'norm_gen_prob']
train_data = []
dev_data = []
test_data = []
for elem in dataset["test"]:
    task_id = "Mbpp/{}".format(elem["task_id"])
    base_input_counts = max([len(e["base_details"]) for e in eval_results["eval"][task_id]])
    base_input_counts = max([1, base_input_counts])
    plus_input_counts = max([len(e["plus_details"]) for e in eval_results["eval"][task_id]])
    plus_input_counts = max([1, plus_input_counts])
    metadata = {"text": problems[task_id]["text"]}
    gold_program = {
        "code": problems[task_id]["canonical_solution"], 
        "exec_match": 1, 
        "linear_score": 1.0, 
        "exec_result": [], 
        "gen_prob": 0, 
        "norm_gen_prob": 0}
    generated_programs = [
        {
            "code": e["solution"],
            #"exec_match": (e["base_status"] == e["plus_status"] == "pass"),
            "exec_match": (e["base_status"] == "pass"),
            #"linear_score": (sum(e["base_details"]) + sum(e["plus_details"])) / (base_input_counts + plus_input_counts),
            "linear_score": sum(e["base_details"]) / base_input_counts,
            "exec_result": [],
            "gen_prob": sum(log_probs[task_id.replace("/","_")][int(e["solution_id"])]),
            "norm_gen_prob": sum(log_probs[task_id.replace("/","_")][int(e["solution_id"])])/len(log_probs[task_id.replace("/","_")][int(e["solution_id"])]),
        } for e in eval_results["eval"][task_id]]
    if int(elem["task_id"]) > 600: # this is for the training partition
        train_data.append({"metadata": metadata,
                           "gold_program": gold_program,
                           "generated_programs": generated_programs,})
    elif int(elem["task_id"]) > 510 and int(elem["task_id"]) <=600: # this is for the development partition
        dev_data.append({"metadata": metadata,
                           "gold_program": gold_program,
                           "generated_programs": generated_programs,})
    elif int(elem["task_id"]) > 10 and int(elem["task_id"]) <= 510: # this is for the test partition
        test_data.append({"metadata": metadata,
                           "gold_program": gold_program,
                           "generated_programs": generated_programs,})

In [47]:
# save a list of dictionaries to jsonl file
prefix = "/mnt/scratch/haausing/code_reranking/code/lever/data/mbpp/"
import json
with open(prefix + "deepseek-coder-7b-instruct-v1.5_temp_0.8_base_train.jsonl", "w") as f:
    for elem in train_data:
        f.write(json.dumps(elem) + "\n")
with open(prefix + "deepseek-coder-7b-instruct-v1.5_temp_0.8_base_dev.jsonl", "w") as f:
    for elem in dev_data:
        f.write(json.dumps(elem) + "\n")
with open(prefix + "deepseek-coder-7b-instruct-v1.5_temp_0.8_base_test.jsonl", "w") as f:
    for elem in test_data:
        f.write(json.dumps(elem) + "\n")

In [ ]:
str(type(0.5))

<class 'float'>
